<a href="https://colab.research.google.com/github/kikjeck1/Final-project-DS/blob/master/tink_NLP_model(text_generation).ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Модель получилась не оформленной. Помешал недостаток опыта работы с NLP моделями, нейросетями и с сохранением модели. Я использую для обучения статьи английской википедии, так как модель потребляет очень много памяти приходится её учить не на самых больших объёмах. У класса метод fit() работает хорошо. У метода train есть параметр method от чего реализуется либо k-gramm способ генерации либо k-gramm + classic ML. Модель получилась не очень гибкой и желательно задавать ровно 3 стартовых слова. Пример обучения и кода приведён ниже, под кодом модели. 

In [1]:
import numpy as np
import pandas as pd
import re
import math
from sklearn.ensemble import RandomForestClassifier
import sklearn
import nltk
from nltk.tokenize import word_tokenize
from string import punctuation
from sklearn.feature_extraction.text import CountVectorizer
from gensim.models.word2vec import Word2Vec
from sklearn.preprocessing import LabelEncoder


In [2]:
!pip install datasets
from datasets import load_dataset

ds = load_dataset("wikipedia", "20220301.simple")

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


  0%|          | 0/1 [00:00<?, ?it/s]

In [29]:
# model = Word2Vec(sentences=snts, size=100, window=10, min_count=10)


class NLP_model:
  """
  init
  fit
  generate"""

  def __init__(self):
    self.d = None
    self.v2w_model = None
    self.snts = None

  def fit(self, text):

    def tokenize(text, nump=False):
      sentences = []

      for art in text:
        # snts = re.split(";|.|!|?", art)
        snts = art.split('.')
    #     print(snts)
        for snt in snts:
          if snt != '': sentences.append(snt.strip())

      # print(sentences5[:10])
      for i in range(len(sentences)):
        snt = sentences[i].split()
        a = []
        for w in snt:
          c = w.rstrip(' ' + punctuation).lower().lstrip(' ' + punctuation)
          if c != '':
            a.append(c)

        # a.append(".") #!!!!!!
        if nump:
          sentences[i] = np.array(a) # add type of string??
        else:
          sentences[i] = a
      if nump:
        return np.array(sentences)
      else:
        return sentences


    def kgramm(k: int, sentences: list) -> dict:
    
      dt = dict()
      for snt in sentences:
          if len(snt) <= k:
              continue
          
          grams = np.lib.stride_tricks.sliding_window_view(snt, k)
          for i in range(len(grams) - 1):
              tup = tuple(grams[i])
              if tup not in dt:
                  dt[tup] = {grams[i+1][-1] : 1}
              else:
                  if grams[i+1][-1] in dt[tup]:
                      dt[tup][grams[i+1][-1]] += 1
                  else:
                      dt[tup][grams[i+1][-1]] = 1
                  
      return dt

    self.snts = tokenize(text)
    self.d = [kgramm(i, self.snts[:10**5]) for i in range(3, 0, -1)]
    self.v2w_model = Word2Vec(sentences=self.snts[:10**5], size=40, window=5, min_count=0)


  def generate(self, prefix=["it", "was", "a"], method="kgramm", n_words = 5):

    def test(d, words, n=5):
      # level = len(words)
      last = tuple(words)
      print(*last)
      j = 0
      i = 0
      while i < n:
        try:
          values = np.fromiter(d[j][last[j:]].values(), dtype=int)
          prob = values / np.sum(values)
          a = np.fromiter(d[j][last[j:]].keys(), dtype="U16")
          last = (*last[1:], np.random.choice(a, p=prob))
          # print(last[-1],f"({3-j})", end=" ")
          print(last[-1], end=" ")
          i += 1
        except KeyError:
          j += 1
          if j == 3:
            raise ValueError("Модель не может подобрать продолжение.\
            Попробуйте ещё раз или измените префикc.")

    def ngramm_forest(dcts,snts, start=prefix, word_pos=4):
      # tail = start.split()
      tail = start
      model = self.v2w_model
      gram_ind = 0
      d = dcts
      if tuple(tail) not in d[gram_ind]:
        tail = tail[1:]
        gram_ind += 1
      
      # print(tail)
      # print(gram_ind)
      b = np.array(sorted(d[gram_ind][tuple(tail)].items(), key=lambda x: x[1], reverse=True)[:3])
      st = set(b[:, 0])
      a = []
      for snt in snts[:10**5]: # np random choice in case of too long working time
        if len(a) > 5000:
            break
        for i in range(3, len(snt)):
          if snt[i] in st and tail[0] != snt[i-3 + gram_ind]:
            a.append([*model.wv[snt[i-3]],
                    len(snt[i-3]),
                    *model.wv[snt[i-2]],
                    len(snt[i-2]),
                    *model.wv[snt[i-1]],
                    len(snt[i-1]),
                    i,
                    snt[i]])
        
      a = np.array(a)
      # print(a.shape)
      # print(a[:4, :])
      X = np.delete(a, -1, 1)
      y = a[:, -1]
      le = LabelEncoder()
      le.fit(y)
      y = le.transform(y)
      # print("started fitting the model")
      forest = RandomForestClassifier(max_depth=5, random_state=0, n_estimators=10)
      forest.fit(X, y)
      next_word = le.inverse_transform(forest.predict([[*model.wv[prefix[0]],
                  len(prefix[0]),
                  *model.wv[prefix[1]],
                  len(prefix[1]),
                  *model.wv[prefix[2]],
                  len(prefix[2]),
                  word_pos]]))[0]
      return next_word


    if method == "kgramm":
      test(self.d, words=prefix, n=n_words)
    elif method == "forest":
      tail = prefix
      print(*tail, end=" ")
      for i in range(n_words):
        next = ngramm_forest(self.d, snts=self.snts, start=tail, word_pos=4+i)
        print(next, end=" ")
        tail = [*tail[1:], next]
    else:
      raise KeyError
    

In [30]:
mod = NLP_model()
text = ds["train"]["text"]
mod.fit(text)

In [32]:
mod.generate(method="forest", prefix=["he", "could", "build"], n_words=10)

he could build a bridge because the water is about 30 meters deep 

In [49]:
mod.generate(method="kgramm", prefix=["he", "could", "build"], n_words=10)

he could build
a heavy smoker george died of cold and wet things 

In [4]:
# def test(d, words, n=5):
#   # level = len(words)
#   last = tuple(words.split())
#   print(*last)
#   j = 0
#   i = 0
#   while i < n:
#     try:
#       values = np.fromiter(d[j][last[j:]].values(), dtype=int)
#       prob = values / np.sum(values)
#       a = np.fromiter(d[j][last[j:]].keys(), dtype="U16")
#       last = (*last[1:], np.random.choice(a, p=prob))
#       print(last[-1],f"({3-j})", end=" ")
#       i += 1
#     except KeyError:
#       j += 1
#       if j == 3:
#         raise ValueError("Модель не может подобрать продолжение. Попробуйте ещё раз или измените префикc.")

        
        
# def kgramm(k: int, sentences: list) -> dict:
    
#   dt = dict()
#   for snt in sentences:
#       if len(snt) <= k:
#           continue
      
#       grams = np.lib.stride_tricks.sliding_window_view(snt, k)
#       for i in range(len(grams) - 1):
#           tup = tuple(grams[i])
#           if tup not in dt:
#               dt[tup] = {grams[i+1][-1] : 1}
#           else:
#               if grams[i+1][-1] in dt[tup]:
#                   dt[tup][grams[i+1][-1]] += 1
#               else:
#                   dt[tup][grams[i+1][-1]] = 1
#   return dt

# def tokenize(text, nump=True):
#   sentences = []

#   for art in text:
#     # snts = re.split(";|.|!|?", art)
#     snts = art.split('.')
# #     print(snts)
#     for snt in snts:
#       if snt != '': sentences.append(snt.strip())

#   # print(sentences5[:10])
#   for i in range(len(sentences)):
#     snt = sentences[i].split()
#     a = []
#     for w in snt:
#       c = w.rstrip(' ' + punctuation).lower().lstrip(' ' + punctuation)
#       if c != '':
#         a.append(c)

#     # a.append(".") #!!!!!!
#     if nump:
#       sentences[i] = np.array(a) # add type of string??
#     else:
#       sentences[i] = a
#   if nump:
#     return np.array(sentences)
#   else:
#     return sentences

# def extract_features(sentences, model): # word2vec model
#   features = []
#   for snt in sentences:
#     if len(snt) <= 3:
#       continue
    
    
#     for i in range(3, len(snt)):
#       row = []



In [33]:
# text = ds["train"]["text"]
# # sentences = tokenize(text)
# # sentences[:2]
# text = ds["train"]["text"]
# snts = tokenize(text, nump=False)

In [34]:
# snts[0]

In [35]:
# d = [kgramm(i, np.random.choice(sentences, size=10**5)) for i in range(3, 0, -1)]
# d = [kgramm(i, snts[:2*10**5]) for i in range(3, 0, -1)]
# d = [kgramm(3, snts[:2*10**5]),]

In [36]:

# d[0][tuple("on the same".split())].items()
# b = np.array(sorted(d[0][tuple("on the same".split())].items(), key=lambda x: x[1], reverse=True)[:3])
# set(b[:, 0])
# d.append()

In [37]:
# for i in range(4):
#   test(d, "it was a", n = 10)
#   print()

In [39]:
# !pip install gensim
# from gensim.models.word2vec import Word2Vec
# model = Word2Vec(sentences=snts, size=100, window=10, min_count=10)
# model = Word2Vec(sentences=snts[:2*10**5], size=40, window=5, min_count=0)


In [ ]:
# from gensim.models.word2vec import Word2Vec
# model.save("word2vec.model")

In [23]:
# from gensim.models.word2vec import Word2Vec
# model = Word2Vec.load("word2vec.model")

In [36]:
# tail = ["it", "was", "a"]
# b = np.array(sorted(d[0][tuple(tail)].items(), key=lambda x: x[1], reverse=True))[:3]
# model["."]

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:3: DeprecationWarning: Call to deprecated `__getitem__` (Method will be removed in 4.0.0, use self.wv.__getitem__() instead).
  This is separate from the ipykernel package so we can avoid doing imports until


array([-0.19890684, -1.261631  ,  0.04040638, -1.4906827 , -1.7610592 ,
       -1.1968117 ,  1.4301373 ,  1.5252349 ,  0.619835  , -1.0439737 ,
        3.0477262 ,  1.837472  ,  2.5491025 ,  0.96900696,  1.9693129 ,
        0.38344952, -0.06435214, -0.5196934 , -0.6399106 ,  0.4739755 ,
        3.0627694 , -1.2734512 ,  0.7570529 , -0.02657134, -0.41007715,
       -1.9053028 ,  2.0340095 ,  0.83900607,  4.1440396 ,  2.472197  ,
        0.6530064 ,  2.3352132 , -1.1873986 ,  0.43339884, -2.5698922 ,
       -0.60294354,  1.7513331 , -1.1444054 ,  1.3005747 ,  2.719398  ],
      dtype=float32)

In [41]:
# from sklearn.model_selection import train_test_split
# from sklearn.preprocessing import LabelEncoder
# import gc

# def ngramm_forest(dcts, start=["it", "was", "a"]):
#   # tail = start.split()
#   tail = start

#   gram_ind = 0
#   d = dcts
#   if tuple(tail) not in d[gram_ind]:
#     tail = tail[1:]
#     gram_ind += 1
  
#   print(tail)
#   print(gram_ind)
#   b = np.array(sorted(d[gram_ind][tuple(tail)].items(), key=lambda x: x[1], reverse=True)[:3])
#   st = set(b[:, 0])
#   a = []
#   for snt in snts[:2*10**5]: # np random choice in case of too long working time
#     if len(a) > 5000:
#         break
#     for i in range(3 - gram_ind, len(snt)):
#       if snt[i] in st and tail[0] != snt[i-3 + gram_ind]:
#         a.append([*model.wv[snt[i-3]],
#                 len(snt[i-3]),
#                 *model.wv[snt[i-2]],
#                 len(snt[i-2]),
#                 *model.wv[snt[i-1]],
#                 len(snt[i-1]),
#                 i,
#                 snt[i]])
    
#   a = np.array(a)
#   # print(a.shape)
#   # print(a[:4, :])
#   X = np.delete(a, -1, 1)
#   y = a[:, -1]
#   le = LabelEncoder()
#   le.fit(y)
#   y = le.transform(y)
#   # print("started fitting the model")
#   forest = RandomForestClassifier(max_depth=5, random_state=0, n_estimators=10)
#   forest.fit(X, y)

#   next_word = le.inverse_transform(forest.predict([[*model.wv[tail[0]],
#               len(tail[0]),
#               *model.wv[tail[1]],
#               len(tail[1]),
#               *model.wv[tail[2]],
#               len(tail[2]),
#               4]]))[0]
#   return next_word

#   # start = [*start[1:],next_word]
#   # gc.collect()
#   # return ngramm_forest(dcts, start)
  



In [42]:
# tail = ["he", "lost", "a"]
# for i in range(5):
#   next = ngramm_forest(d, start=tail)
#   print(next)
#   tail = [*tail[1:], next]

In [43]:
# tail = ["it", "was", "a"]
# b = np.array(sorted(d[0][tuple(tail)].items(), key=lambda x: x[1], reverse=True)[:3])

# a = []
# st = set(b[:, 0])
# for snt in snts:
#   for i in range(3, len(snt)):
#     if snt[i] in st and tail[0] != snt[i-3]:
#       a.append([*model.wv[snt[i-3]],
#               len(snt[i-3]),
#               *model.wv[snt[i-2]],
#               len(snt[i-2]),
#               *model.wv[snt[i-1]],
#               len(snt[i-1]),
#               i,
#               snt[i]])

In [44]:
# len([*model.wv[snt[i-3]],
#               len(snt[i-3]),
#               *model.wv[snt[i-2]],
#               len(snt[i-2]),
#               *model.wv[snt[i-1]],
#               len(snt[i-1]),
#               i,
#               snt[i]])

In [45]:
# from sklearn.model_selection import train_test_split
# from sklearn.preprocessing import LabelEncoder

# le = LabelEncoder()
# df = pd.DataFrame(a)

# X = df.drop(columns=[304])
# y = df[304]
# le.fit(y)
# y = le.transform(y)
# print(y)
# X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.33)

# forest = RandomForestClassifier(max_depth=20, random_state=0, n_estimators=10)

# forest.fit(X_train, y_train)


In [46]:
# from sklearn.metrics import accuracy_score
# # test_ML(forest)
# # forest.predict(X.iloc[:2, :])
# y_pred = forest.predict(X_test)
# accuracy_score(y_test, y_pred)

In [47]:
# le.inverse_transform(forest.predict([[*model.wv[tail[0]],
#               len(tail[0]),
#               *model.wv[tail[1]],
#               len(tail[1]),
#               *model.wv[tail[2]],
#               len(tail[2]),
#               4]]))

In [48]:
# a = []
# # np.random.choice(sentences, size=10)
# for snt in snts[:1000]:
#   n = len(snt)
#   # print(snt)
#   for i in range(3, n):
#     a.append([*model.wv[snt[i-3]],
#               len(snt[i-3]),
#               *model.wv[snt[i-2]],
#               len(snt[i-2]),
#               *model.wv[snt[i-1]],
#               len(snt[i-1]),
#               i,
#               *model.wv[snt[i]]])
